#  PREPARAZIONE DEL DATASET

In [2]:
# Import necessari
import os
import re
import pandas as pd

# Dataset
df = pd.read_csv("Dataset/Movies.csv")

# Subsampling del Dataset
df = df.sample(n=100, random_state=42, replace=False)
df = df[['Title', 'Plot']].copy()
df = df.reset_index(drop=True)
 
# Crea la cartella per i .txt
directory = "Files"
os.makedirs(directory, exist_ok=True)

# Crea file di testo per ogni film:
# nome file basato sul Title e contenuto su Plot
for idx, row in df.iterrows():
    movieTitle = row["Title"]
    moviePlot = row["Plot"]
    
    filePath = os.path.join(directory, f"{movieTitle}.txt")

    with open(filePath, "w", encoding="utf-8") as f:
        f.write(str(moviePlot))

# ELASTICSEARCH

In [ ]:
import os
import time
from dotenv import load_dotenv
from elasticsearch import Elasticsearch, helpers # type: ignore

# Nome dell'indice
INDEX_NAME = "moviesindex"

# Carica variabili d'ambiente dal file .env
load_dotenv(dotenv_path="EnvAndDocker/.env")

# Recupera i valori dal .env
URL = os.getenv("URL")
PASSWORD = os.getenv("PASSWORD")

# Definizione del mapping e degli analyzer
MAPPING = {
    "settings": {
        "analysis": {
            "filter": {
                "english_stop": {"type": "stop", "stopwords": "_english_"},
                "english_stemmer": {"type": "stemmer", "language": "english"},
                "english_possessive_stemmer": {"type": "stemmer", "language": "possessive_english"},
            },
            "analyzer": {
                "standard_custom": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "english_possessive_stemmer",
                        "lowercase",
                        "asciifolding",
                        "english_stop",
                        "english_stemmer",
                    ],
                }
            },
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "standard",
                "search_analyzer": "standard",
            },
            "content": {
                "type": "text",
                "analyzer": "standard_custom",
                "search_analyzer": "standard_custom",
            },
        }
    },
}


def build_index():
    """Crea l'indice moviesindex e indicizza i file .txt."""

    # Connessione a Elasticsearch
    es = Elasticsearch(URL, basic_auth=("elastic", PASSWORD))

    # Verifica connessione
    if not es.ping():
        raise ConnectionError("Impossibile connettersi a Elasticsearch. Controlla URL e password nel .env")

    # Elimina indice se esiste già
    if es.indices.exists(index=INDEX_NAME):
        es.indices.delete(index=INDEX_NAME)
        print(f"Indice esistente '{INDEX_NAME}' eliminato.")

    # Creazione nuovo indice con il mapping definito
    es.indices.create(index=INDEX_NAME, body=MAPPING)
    print(f"Indice '{INDEX_NAME}' creato correttamente.")

    # Lista di documenti da indicizzare
    actions = []
    for filename in os.listdir("Files"):
        if not filename.endswith(".txt"):
            continue
        path = os.path.join("Files", filename)
        with open(path, "r", encoding="utf-8") as f:
            content = f.read()

        actions.append({
            "_index": INDEX_NAME,
            "_source": {
                "title": os.path.splitext(filename)[0],  # nome del file senza estensione
                "content": content
            }
        })

    if not actions:
        print("Nessun file .txt trovato nella cartella.")
        return

    # Indicizzazione in blocco
    start = time.perf_counter()
    success, _ = helpers.bulk(es.options(request_timeout=120), actions)
    elapsed = time.perf_counter() - start

    print(f"Documenti indicizzati correttamente: {success}")
    print(f"Tempo totale: {elapsed:.2f} secondi")

    return elapsed


if __name__ == "__main__":
    build_index()


Indice 'moviesindex' creato correttamente.


/tmp/ipykernel_19281/3807077765.py:102: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  success, _ = helpers.bulk(es, actions, request_timeout=120)


Documenti indicizzati correttamente: 100
Tempo totale: 0.52 secondi
